In [1]:
!python --version

Python 3.9.7


In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("bank_data_prediction_task.csv", low_memory=False)

In [4]:
data.head(10)

Unnamed: 0  age          job  marital    education  default housing loan  \
0           1   37     services  married  high.school       no     yes   no   
1           2   56     services  married  high.school       no      no  yes   
2           3   41  blue-collar  married      unknown  unknown      no   no   
3           4   25     services   single  high.school       no     yes   no   
4           5   25     services   single  high.school       no     yes   no   
5           6   35  blue-collar  married     basic.6y       no     yes   no   
6           7   54      retired  married     basic.9y  unknown     yes  yes   
7           8   46  blue-collar  married     basic.6y  unknown     yes  yes   
8           9   41   technician   single  high.school       no     yes   no   
9          10   37       admin.  married  high.school       no     yes   no   

     contact month  ... pdays  previous     poutcome  emp.var.rate  \
0  telephone   may  ...   999         0  nonexistent           1.1   
1  telephone   may  ...   999         0  nonexistent           1.1   
2  telephone   may  ...   999         0  nonexistent           1.1   
3  telephone   may  ...   999         0  nonexistent           1.1   
4  telephone   may  ...   999         0  nonexistent           1.1   
5  telephone   may  ...   999         0  nonexistent           1.1   
6  telephone   may  ...   999         0  nonexistent           1.1   
7  telephone   may  ...   999         0  nonexistent           1.1   
8  telephone   may  ...   999         0  nonexistent           1.1   
9  telephone   may  ...   999         0  nonexistent           1.1   

   cons.price.idx cons.conf.idx  euribor3m  nr.employed    y  \
0          93.994         -36.4      4.857       5191.0   no   
1          93.994         -36.4      4.857       5191.0   no   
2          93.994         -36.4      4.857       5191.0  yes   
3          93.994         -36.4      4.857       5191.0   no   
4          93.994         -36.4      4.857       5191.0   no   
5          93.994         -36.4      4.857       5191.0   no   
6          93.994         -36.4      4.857       5191.0   no   
7          93.994         -36.4      4.857       5191.0   no   
8          93.994         -36.4      4.857       5191.0   no   
9          93.994         -36.4      4.857       5191.0   no   

   test_control_flag  
0     campaign group  
1     campaign group  
2     campaign group  
3     campaign group  
4     campaign group  
5     campaign group  
6     campaign group  
7     campaign group  
8     campaign group  
9     campaign group  

[10 rows x 23 columns]

# 1. Basic data engineering

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

### Dropping the first column

In [6]:
data = data.iloc[:, 1:]

In [7]:
data['y'] = data['y'].map({'yes': 1, 'no': 0})

In [8]:
cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome', 'y', 'test_control_flag']
data[cat_cols] = data[cat_cols].astype('category')

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   age                41188 non-null  int64   
 1   job                41188 non-null  category
 2   marital            41188 non-null  category
 3   education          41188 non-null  category
 4   default            41188 non-null  category
 5   housing            41188 non-null  category
 6   loan               41188 non-null  category
 7   contact            16476 non-null  category
 8   month              16476 non-null  category
 9   day_of_week        16476 non-null  category
 10  duration           16476 non-null  float64 
 11  campaign           16476 non-null  float64 
 12  pdays              41188 non-null  int64   
 13  previous           41188 non-null  int64   
 14  poutcome           41188 non-null  category
 15  emp.var.rate       41188 non-null  float64 
 16  cons

### Check for missing values

In [10]:
data.isna().sum()

age                      0
job                      0
marital                  0
education                0
default                  0
housing                  0
loan                     0
contact              24712
month                24712
day_of_week          24712
duration             24712
campaign             24712
pdays                    0
previous                 0
poutcome                 0
emp.var.rate             0
cons.price.idx         250
cons.conf.idx            0
euribor3m                0
nr.employed              0
y                        0
test_control_flag        0
dtype: int64

#### Missing data in 250 rows, less than 1% of dataset so can be removed

In [11]:
data = data.dropna(subset = ['cons.price.idx'])

In [12]:
data.shape

(40938, 22)

In [13]:
data_y = data[data['y'] == 1]

In [14]:
data_y.shape[0]

5126

In [15]:
data_y.isna().sum()

age                     0
job                     0
marital                 0
education               0
default                 0
housing                 0
loan                    0
contact              2642
month                2642
day_of_week          2642
duration             2642
campaign             2642
pdays                   0
previous                0
poutcome                0
emp.var.rate            0
cons.price.idx          0
cons.conf.idx           0
euribor3m               0
nr.employed             0
y                       0
test_control_flag       0
dtype: int64

### Previous campaign effectiveness

In [16]:
percentage = 2642/data_y.shape[0]*100
print(f"Effectiveness of previous campaingn: {percentage:.2f}%")

Effectiveness of previous campaingn: 51.54%


In [17]:
data.shape

(40938, 22)

In [18]:
# Creating categorical and continuous variable list
cat_var = ["blue","dual_sim","four_g","three_g","touch_screen","wifi"]
con_var = ['px_height', 'sc_h', 'sc_w', 'clock_speed', 'battery_power', 'int_memory', 'talk_time', 'pc',
           'n_cores', 'px_width', 'fc', 'mobile_wt', 'm_dep', 'ram']

### Correlation heatmap

In [19]:
correlation_matrix = data.corr()

plt.figure(figsize=(30, 30))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()

In [20]:
# Create a set to keep track of columns to drop
columns_to_drop = set()

# Iterate through the upper triangular part of the correlation matrix
for i in range(len(correlation_matrix.columns)):
    for j in range(i + 1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.9:
            columns_to_drop.add(correlation_matrix.columns[j])

# Drop the selected columns
data = data.drop(columns=columns_to_drop)

# Print the columns that were dropped
print("Dropped Columns:", columns_to_drop)

Dropped Columns: {'euribor3m', 'nr.employed'}


### pdays column values

In [21]:
data['pdays'].unique()

array([999,   4,   5,   1,   0,   3,   6,   8,   9,   2,  11,  10,  12,
         7,  13,  14,  15,  16,  21,  18,  26,  19,  20,  17,  22,  25,
        27])

In [22]:
data['was_contacted'] = data['pdays'].apply(lambda row: 0 if row == 999 else 1)
data[['pdays','was_contacted']].tail()

pdays  was_contacted
41183      9              1
41184    999              0
41185    999              0
41186    999              0
41187    999              0

### month column

In [23]:
months = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec': 12}
data['month'] = data['month'].map(months)

In [24]:
data['month'].unique()

[5, 6, 7, 8, 10, ..., 12, 3, 4, 9, NaN]
Length: 11
Categories (10, int64): [4, 8, 12, 7, ..., 5, 11, 10, 9]

In [25]:
data_all = data.drop(columns = ['contact', 'month', 'day_of_week', 'duration', 'campaign']) ## remove all campaign columns

## Save data_all data frame for further processing

In [26]:
data_all.shape

(40938, 16)

In [27]:
data_all.to_csv('all_data.csv', index=False)

## Save campaign data for further analysis

In [28]:
data_campaign = data[data['test_control_flag'] == 'campaign group']

In [29]:
data_campaign.shape

(16476, 21)

In [30]:
data_campaign.to_csv('campaign_data.csv', index=False)

### column reductions for reduced data

In [31]:
data['is_default'] = data['default'].apply(lambda row: 1 if row == 'yes' else 0)

In [32]:
data['is_housing'] = data['housing'].apply(lambda row: 1 if row == 'yes' else 0)

In [33]:
data['is_loan'] = data['loan'].apply(lambda row: 1 if row == 'yes' else 0)

In [34]:
data.shape

(40938, 24)

### drop campaign columns

In [35]:
data_all_corr_dropped = data.drop(columns = ['contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays']) # campaign columns

data_all_corr_dropped.shape

(40938, 18)

## Save reduced data frame

In [36]:
data_all_corr_dropped.to_csv('reduced_data.csv', index=False)

In [37]:
missing_columns = set(data_all.columns) - set(data_all_corr_dropped.columns)
print("Missing columns in data_all_corr_dropped:", missing_columns)

Missing columns in data_all_corr_dropped: {'pdays'}


In [38]:
missing_columns = set(data_all_corr_dropped.columns) - set(data_all.columns)
print("Missing columns in data_all:", missing_columns)

Missing columns in data_all: {'is_default', 'is_loan', 'is_housing'}


In [39]:
# Print the differing column names
differing_column = set(data_all_corr_dropped.columns) ^ set(data_all.columns)


differing_column

{'is_default', 'is_housing', 'is_loan', 'pdays'}